<a href="https://colab.research.google.com/github/RownakTahmid/cypher10-cnn-optimization/blob/master/cnn_on_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing important libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch as t
import torch.nn as nn
from torch.utils.data import DataLoader as dL
from tqdm import tqdm
import torch.optim as optim

In [ ]:
pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# Torchvision includes several image datasets that can be used for training and testing neural networks.
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchmetrics

In [ ]:
transform = transforms.ToTensor()

In [ ]:
batch_size = 64
train_dataset = CIFAR10(root="dataset/", download = True, transform = transform, train = True)
train_loader =  dL(dataset=train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = CIFAR10(root="dataset/", download = True, transform = transform, train = False)
test_loader = dL(dataset=test_dataset, batch_size = batch_size, shuffle = True)

100%|██████████| 170M/170M [00:05<00:00, 30.7MB/s]


In [ ]:
# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# print(images.shape)  # Example: torch.Size([64, 3, 32, 32])


In [ ]:
# from collections import Counter

# all_labels = []
# for _, labels in train_loader:
#     all_labels.extend(labels.tolist())

# counts = Counter(all_labels)
# print(counts)


In [ ]:
# # single images
# d = iter(train_loader)
# image, label = next(d)
# print(image.shape)
# for i in range(5):
#     img = image[i].permute(1, 2, 0)  # Convert from [C, H, W] to [H, W, C]
#     plt.imshow(img)
#     plt.title(f"Label: {label[i]}")
#     plt.axis('off')
#     plt.show()


In [ ]:
# # grid images
# def imshow(img):
#    npimg = img.numpy()
#    plt.imshow(np.transpose(npimg, (1, 2, 0)))
#    plt.show()

# # get some random training images
# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# labels
# # show images
# imshow(torchvision.utils.make_grid(images))


In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channels, num_classes):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size = 3, stride=1, padding= 1)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16,32,3, padding= 1)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32,64,3, padding= 1)
    self.bn3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64,128,3, padding= 1)
    self.bn4 = nn.BatchNorm2d(128)
    self.conv5 = nn.Conv2d(128,256,3, padding= 1)
    self.bn5 = nn.BatchNorm2d(256)
    self.conv6 = nn.Conv2d(256,512,3, padding= 1)
    self.bn6 = nn.BatchNorm2d(512)
    # self.conv7 = nn.Conv2d(512,1024,3, padding= 1)
    # self.bn7 = nn.BatchNorm2d(1024)
    self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding= (1,1))
    self.relu = nn.PReLU()
    self.dropout = nn.Dropout(0.48)
    self.dropout1 = nn.Dropout(0.11)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear( 512*4*4, num_classes)
  def forward(self,x):
    x = self.relu(self.bn1(self.conv1(x)))
    x = self.relu(self.bn2(self.conv2(x)))
    x =self.dropout1(x)
    x = self.pool(x)
    x = self.relu(self.bn3(self.conv3(x)))
    x = self.relu(self.bn4(self.conv4(x)))
    x =self.dropout1(x)
    x = self.pool(x)
    x = self.relu(self.bn5(self.conv5(x)))
    x = self.relu(self.bn6(self.conv6(x)))
    # x = self.relu(self.bn7(self.conv7(x)))
    x = self.pool(x)
    x = self.dropout(self.flatten(x))
    x = self.fc1(x)
    # x = self.relu(self.conv1(x))
    # x = self.pool(x)
    # x = self.relu(self.conv2(x))
    # x = self.pool(x)
    # x = self.relu(self.conv3(x))
    # x = self.pool(x)
    # x = self.relu(self.conv4(x))
    # # x = self.pool1(x)
    # x = self.flatten(x)
    # x = self.fc1(x)
    return x


In [ ]:
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
model = CNN(in_channels=3, num_classes=10).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.SGD(model.parameters(), momentum= 0.9 , lr=0.01, weight_decay=4e-4)
# scheduler = t.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [ ]:
# Variables to store the best model
minimum_val_loss = float('inf')
best_model = None
# TO track losses
training_losses = []
val_losses = []


epochs = 15
for epoch in range(epochs):
  print(f"Epoch [{epoch + 1}/{epochs}]")
  model.train()
  running_losses = 0.0
  # , desc= f"Epoch {epoch+1}/{epochs}"
  for images, labels in tqdm(train_loader ):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss:{loss.item():.4f}") #loss.item()converts the loss tensor into a standard python number
    running_losses += loss.item()
  avg_loss = running_losses/len(train_loader)
  training_losses.append(avg_loss)
  print(f"Epoch [{epoch+1}/{epochs}], Loss:{avg_loss:.4f}")

  model.eval()
  running_val_loss = 0.0
  with t.no_grad():
      for images, labels in tqdm(test_loader, desc="Validation"):
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)
          running_val_loss += loss.item()

  avg_val = running_val_loss / len(test_loader)
  val_losses.append(avg_val)
  print(f"  → Avg Validation Loss: {avg_val:.4f}")

  if avg_val < minimum_val_loss:
        minimum_val_loss = avg_val
        best_model = model.state_dict()
        print("  → New best model saved!")



Epoch [1/15]


  0%|          | 1/782 [00:02<27:53,  2.14s/it]

Epoch [1/15], Loss:2.6471


  0%|          | 2/782 [00:03<20:25,  1.57s/it]

Epoch [1/15], Loss:4.7442


...

100%|█████████▉| 781/782 [16:19<00:01,  1.18s/it]

Epoch [7/15], Loss:0.6653


100%|██████████| 782/782 [16:20<00:00,  1.25s/it]


Epoch [7/15], Loss:0.7565
Epoch [7/15], Loss:0.6450


Validation:  36%|███▌      | 56/157 [00:23<00:39,  2.55it/s]

In [ ]:
# # print(best_model)
# print(minimum_val_loss)

In [ ]:
from torchmetrics import Accuracy, Precision, Recall
model.load_state_dict(best_model)
# Instantiate metrics
acc = Accuracy(task="multiclass", num_classes=10).to(device)
precision = Precision(task="multiclass", num_classes=10).to(device)
recall = Recall(task="multiclass", num_classes=10).to(device)
# Set model to eval mode
model.eval()

# Reset metrics (important if reused)
acc.reset()
precision.reset()
recall.reset()

with t.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predictions = t.max(outputs, 1)  # Get predicted class indices
        acc.update(predictions, labels)
        precision.update(predictions, labels)
        recall.update(predictions, labels)

# Compute final scores
test_acc = acc.compute()
test_precision = precision.compute()
test_recall = recall.compute()

print(f"Test Accuracy : {test_acc:.4f}")
print(f"Precision     : {test_precision:.4f}")
print(f"Recall        : {test_recall:.4f}")


In [ ]:
print("Train losses:", training_losses)
print("Val losses:", val_losses)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(training_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs. Validation Loss')
plt.legend()
plt.show()


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
path = '/content/drive/MyDrive/pytorch/cifar10-6C-3P-3 dropouts-sgd-0.01-lrscheduler-0.9momn-decay5e.pt'
t.save(model.state_dict(), path)

Mounted at /content/drive


In [ ]:
# from google.colab import files
# files.download(path)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
# from PIL import Image, ImageOps
# import torchvision.transforms as transforms
# image_path = '/content/drive/MyDrive/pytorch/digit_images/9.png'
# # Load image
# img = Image.open(image_path).convert("L")  # Grayscale
# img = ImageOps.invert(img)                        # Invert image (dark digit on light bg)
# img = img.resize((28, 28))  # MNIST size

# # Transform to Tensor
# transform = transforms.Compose([
#     transforms.ToTensor() # MNIST normalization
# ])
# transforms.Normalize((0.1307,), (0.3081,))
# img_tensor = transform(img).unsqueeze(0)  # shape: [1, 1, 28, 28]


In [ ]:
# import matplotlib.pyplot as plt

# plt.subplot(1, 2, 1)
# plt.title("Original (grayscale)")
# plt.imshow(Image.open(image_path).convert("L"), cmap="bwr")

# plt.subplot(1, 2, 2)
# plt.title("After Inversion")
# plt.imshow(img, cmap="gray")
# plt.show()

In [ ]:
# model.eval()

# # Move to GPU if available
# device = t.device('cuda' if t.cuda.is_available() else 'cpu')
# model.load_state_dict(t.load(path))
# model = model.to(device)
# img_tensor = img_tensor.to(device)

# # Predict
# with t.no_grad():
#     output = model(img_tensor)
#     prediction = output.argmax(dim=1, keepdim=True)

# print("Predicted Digit:", prediction.item())
